In [4]:
def compare_phonemes(word, correct_phonemes, input_phonemes, mapping):
    """
    So sánh input phonemes với correct phonemes, xử lý trường hợp thiếu/thừa âm.
    
    Args:
        word: Từ gốc (str)
        correct_phonemes: Chuỗi phoneme đúng (str) - có khoảng trắng giữa các âm
        input_phonemes: Chuỗi phoneme input (str) - có khoảng trắng giữa các âm
        mapping: List của tuple (grapheme, phoneme) - ví dụ [("J", "d͡ʒ"), ("a", "æ"), ...]
    
    Returns:
        List các lỗi, mỗi lỗi là dict {
            'grapheme': phần chữ cái sai,
            'expected': phoneme đúng,
            'actual': phoneme nhận được (hoặc 'missing' nếu thiếu)
        }
    """
    correct_list = correct_phonemes.split()
    input_list = input_phonemes.split()
    
    # Xây dựng reverse mapping từ phoneme về grapheme
    phoneme_to_grapheme = {}
    grapheme_positions = []
    pos = 0
    
    for grapheme, phoneme in mapping:
        phoneme_to_grapheme[phoneme] = grapheme
        grapheme_positions.append((grapheme, phoneme, pos, pos + len(grapheme)))
        pos += len(grapheme)
    
    # Dynamic Programming để tìm alignment tốt nhất
    n, m = len(correct_list), len(input_list)
    
    # dp[i][j] = (cost, path) - cost là số lỗi, path để trace back
    dp = [[None for _ in range(m + 1)] for _ in range(n + 1)]
    
    # Base cases
    dp[0][0] = (0, [])
    
    # Thiếu âm từ đầu (input bị thiếu)
    for i in range(1, n + 1):
        dp[i][0] = (i, dp[i-1][0][1] + [('missing', i-1)])
    
    # Thừa âm từ đầu (input có thừa)
    for j in range(1, m + 1):
        dp[0][j] = (j, dp[0][j-1][1] + [('extra', j-1)])
    
    # Fill DP table
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            options = []
            
            # Match hoặc mismatch
            cost = 0 if correct_list[i-1] == input_list[j-1] else 1
            options.append((
                dp[i-1][j-1][0] + cost,
                dp[i-1][j-1][1] + [('match' if cost == 0 else 'mismatch', i-1, j-1)]
            ))
            
            # Delete (thiếu âm trong input)
            options.append((
                dp[i-1][j][0] + 1,
                dp[i-1][j][1] + [('missing', i-1)]
            ))
            
            # Insert (thừa âm trong input)
            options.append((
                dp[i][j-1][0] + 1,
                dp[i][j-1][1] + [('extra', j-1)]
            ))
            
            dp[i][j] = min(options, key=lambda x: x[0])
    
    # Trace back để tìm lỗi
    path = dp[n][m][1]
    errors = []
    
    for action in path:
        if action[0] == 'mismatch':
            correct_idx = action[1]
            input_idx = action[2]
            
            expected_phoneme = correct_list[correct_idx]
            actual_phoneme = input_list[input_idx]
            
            # Tìm grapheme tương ứng
            grapheme = phoneme_to_grapheme.get(expected_phoneme, '?')
            
            errors.append({
                'grapheme': grapheme,
                'expected': expected_phoneme,
                'actual': actual_phoneme
            })
        
        elif action[0] == 'missing':
            correct_idx = action[1]
            expected_phoneme = correct_list[correct_idx]
            grapheme = phoneme_to_grapheme.get(expected_phoneme, '?')
            
            errors.append({
                'grapheme': grapheme,
                'expected': expected_phoneme,
                'actual': 'missing'
            })
        
        # Không báo lỗi 'extra' vì có thể người dùng phát âm thêm
    
    return errors


# Test với ví dụ của bạn
word = "January"
correct_phonemes = "d͡ʒ æ n j u ɛ ɹ i"
input_phonemes = "d͡ʒ d͡ʒ ɛ n j d i"
mapping = [
    ("J", "d͡ʒ"),
    ("a", "æ"),
    ("nu", "nju"),
    ("a", "ɛ"),
    ("ry", "ɹi")
]

# Chuyển mapping thành flat version cho dễ xử lý
flat_mapping = [
    ("J", "d͡ʒ"),
    ("a", "æ"),
    ("n", "n"),
    ("u", "j"),
    ("", "u"),
    ("a", "ɛ"),
    ("r", "ɹ"),
    ("y", "i")
]

errors = compare_phonemes(word, correct_phonemes, input_phonemes, flat_mapping)

print("=== Kết quả phân tích ===")
print(f"Từ: {word}")
print(f"Phoneme đúng: {correct_phonemes}")
print(f"Phoneme nhận: {input_phonemes}")
print(f"\nSố lỗi: {len(errors)}")

if errors:
    print("\nChi tiết lỗi:")
    for i, err in enumerate(errors, 1):
        print(f"{i}. Grapheme '{err['grapheme']}':")
        print(f"   - Mong đợi: {err['expected']}")
        print(f"   - Nhận được: {err['actual']}")
else:
    print("\nKhông có lỗi!")


# Test case 2: Thiếu âm giữa
print("\n" + "="*50)
print("Test case 2: Thiếu âm ở giữa")
correct_phonemes2 = "d͡ʒ æ n j u"
input_phonemes2 = "d͡ʒ æ u"  # Thiếu "n j"
flat_mapping2 = [
    ("J", "d͡ʒ"),
    ("a", "æ"),
    ("n", "n"),
    ("u", "j"),
    ("", "u")
]

errors2 = compare_phonemes("Janu", correct_phonemes2, input_phonemes2, flat_mapping2)
print(f"Phoneme đúng: {correct_phonemes2}")
print(f"Phoneme nhận: {input_phonemes2}")
print(f"Số lỗi: {len(errors2)}")
for err in errors2:
    print(f"- {err['grapheme']}: expected '{err['expected']}', got '{err['actual']}'")

=== Kết quả phân tích ===
Từ: January
Phoneme đúng: d͡ʒ æ n j u ɛ ɹ i
Phoneme nhận: d͡ʒ d͡ʒ ɛ n j d i

Số lỗi: 4

Chi tiết lỗi:
1. Grapheme 'a':
   - Mong đợi: æ
   - Nhận được: ɛ
2. Grapheme '':
   - Mong đợi: u
   - Nhận được: missing
3. Grapheme 'a':
   - Mong đợi: ɛ
   - Nhận được: missing
4. Grapheme 'r':
   - Mong đợi: ɹ
   - Nhận được: d

Test case 2: Thiếu âm ở giữa
Phoneme đúng: d͡ʒ æ n j u
Phoneme nhận: d͡ʒ æ u
Số lỗi: 2
- n: expected 'n', got 'missing'
- u: expected 'j', got 'missing'


In [2]:
def compare_phonemes(word, correct_phonemes, input_phonemes, mapping):
    """
    So sánh input phonemes với correct phonemes, xử lý trường hợp thiếu/thừa âm.
    
    Args:
        word: Từ gốc (str)
        correct_phonemes: Chuỗi phoneme đúng (str) - có khoảng trắng giữa các âm
        input_phonemes: Chuỗi phoneme input (str) - có khoảng trắng giữa các âm
        mapping: List của tuple (grapheme, phoneme) - ví dụ [("J", "d͡ʒ"), ("a", "æ"), ("nu", "nju"), ...]
    
    Returns:
        List các lỗi, mỗi lỗi là dict {
            'grapheme': phần chữ cái sai,
            'expected': phoneme đúng,
            'actual': phoneme nhận được (hoặc 'missing' nếu thiếu)
        }
    """
    correct_list = correct_phonemes.split()
    input_list = input_phonemes.split()
    
    # Xây dựng mapping từ phoneme về grapheme (xử lý cả multi-phoneme)
    phoneme_to_grapheme = {}
    correct_idx_to_grapheme = {}
    
    phoneme_idx = 0
    for grapheme, phoneme_str in mapping:
        # Tách phoneme_str thành list các phoneme đơn lẻ
        phonemes = phoneme_str.split()
        
        # Map toàn bộ chuỗi phoneme về grapheme
        phoneme_to_grapheme[phoneme_str] = grapheme
        
        # Map từng phoneme đơn lẻ về grapheme
        for p in phonemes:
            phoneme_to_grapheme[p] = grapheme
            correct_idx_to_grapheme[phoneme_idx] = grapheme
            phoneme_idx += 1
    
    # Dynamic Programming để tìm alignment tốt nhất
    n, m = len(correct_list), len(input_list)
    
    # dp[i][j] = (cost, path) - cost là số lỗi, path để trace back
    dp = [[None for _ in range(m + 1)] for _ in range(n + 1)]
    
    # Base cases
    dp[0][0] = (0, [])
    
    # Thiếu âm từ đầu (input bị thiếu)
    for i in range(1, n + 1):
        dp[i][0] = (i, dp[i-1][0][1] + [('missing', i-1)])
    
    # Thừa âm từ đầu (input có thừa)
    for j in range(1, m + 1):
        dp[0][j] = (j, dp[0][j-1][1] + [('extra', j-1)])
    
    # Fill DP table
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            options = []
            
            # Match hoặc mismatch
            cost = 0 if correct_list[i-1] == input_list[j-1] else 1
            options.append((
                dp[i-1][j-1][0] + cost,
                dp[i-1][j-1][1] + [('match' if cost == 0 else 'mismatch', i-1, j-1)]
            ))
            
            # Delete (thiếu âm trong input)
            options.append((
                dp[i-1][j][0] + 1,
                dp[i-1][j][1] + [('missing', i-1)]
            ))
            
            # Insert (thừa âm trong input)
            options.append((
                dp[i][j-1][0] + 1,
                dp[i][j-1][1] + [('extra', j-1)]
            ))
            
            dp[i][j] = min(options, key=lambda x: x[0])
    
    # Trace back để tìm lỗi
    path = dp[n][m][1]
    errors = []
    
    for action in path:
        if action[0] == 'mismatch':
            correct_idx = action[1]
            input_idx = action[2]
            
            expected_phoneme = correct_list[correct_idx]
            actual_phoneme = input_list[input_idx]
            
            # Tìm grapheme tương ứng từ correct_idx
            grapheme = correct_idx_to_grapheme.get(correct_idx, '?')
            
            errors.append({
                'grapheme': grapheme,
                'expected': expected_phoneme,
                'actual': actual_phoneme
            })
        
        elif action[0] == 'missing':
            correct_idx = action[1]
            expected_phoneme = correct_list[correct_idx]
            grapheme = correct_idx_to_grapheme.get(correct_idx, '?')
            
            errors.append({
                'grapheme': grapheme,
                'expected': expected_phoneme,
                'actual': 'missing'
            })
        
        # Không báo lỗi 'extra' vì có thể người dùng phát âm thêm
    
    return errors


# Test với ví dụ của bạn
word = "January"
correct_phonemes = "d͡ʒ æ n j u ɛ ɹ i"
input_phonemes = "d͡ʒ d͡ʒ ɛ n j d i"
mapping = [
    ("J", "d͡ʒ"),
    ("a", "æ"),
    ("nu", "n j u"),  # Multi-phoneme
    ("a", "ɛ"),
    ("ry", "ɹ i")     # Multi-phoneme
]

errors = compare_phonemes(word, correct_phonemes, input_phonemes, mapping)

print("=== Kết quả phân tích ===")
print(f"Từ: {word}")
print(f"Phoneme đúng: {correct_phonemes}")
print(f"Phoneme nhận: {input_phonemes}")
print(f"\nSố lỗi: {len(errors)}")

if errors:
    print("\nChi tiết lỗi:")
    for i, err in enumerate(errors, 1):
        print(f"{i}. Grapheme '{err['grapheme']}':")
        print(f"   - Mong đợi: {err['expected']}")
        print(f"   - Nhận được: {err['actual']}")
else:
    print("\nKhông có lỗi!")


# Test case 2: Thiếu âm giữa
print("\n" + "="*50)
print("Test case 2: Thiếu âm ở giữa")
correct_phonemes2 = "d͡ʒ æ n j u"
input_phonemes2 = "d͡ʒ æ u"  # Thiếu "n j"
mapping2 = [
    ("J", "d͡ʒ"),
    ("a", "æ"),
    ("nu", "n j u")
]

errors2 = compare_phonemes("Janu", correct_phonemes2, input_phonemes2, mapping2)
print(f"Phoneme đúng: {correct_phonemes2}")
print(f"Phoneme nhận: {input_phonemes2}")
print(f"Số lỗi: {len(errors2)}")
for err in errors2:
    print(f"- {err['grapheme']}: expected '{err['expected']}', got '{err['actual']}'")

=== Kết quả phân tích ===
Từ: January
Phoneme đúng: d͡ʒ æ n j u ɛ ɹ i
Phoneme nhận: d͡ʒ d͡ʒ ɛ n j d i

Số lỗi: 4

Chi tiết lỗi:
1. Grapheme 'a':
   - Mong đợi: æ
   - Nhận được: ɛ
2. Grapheme 'nu':
   - Mong đợi: u
   - Nhận được: missing
3. Grapheme 'a':
   - Mong đợi: ɛ
   - Nhận được: missing
4. Grapheme 'ry':
   - Mong đợi: ɹ
   - Nhận được: d

Test case 2: Thiếu âm ở giữa
Phoneme đúng: d͡ʒ æ n j u
Phoneme nhận: d͡ʒ æ u
Số lỗi: 2
- nu: expected 'n', got 'missing'
- nu: expected 'j', got 'missing'
